# Assignment 3.2
Design a simple multi-agent LLM system where each agent is responsible for a subtask (e.g., planning, summarizing, answering). Agents communicate via message passing using a shared memory structure.

In [1]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

c:\Users\siddh\OneDrive\Desktop\LLM_course\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
llm_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu
Device set to use cpu


### Shared Memory for Message Passing
A simple dictionary is used as shared memory for agent communication.

In [3]:
class SharedMemory:
    def __init__(self):
        self.memory = {}
    def write(self, key, value):
        self.memory[key] = value
    def read(self, key):
        return self.memory.get(key, None)

### Planner Agent
Breaks down the main query into subtasks and writes them to shared memory.

In [4]:
class PlannerAgent:
    def __init__(self, memory):
        self.memory = memory
    def plan(self, question):
        prompt = f"Break the following task into 2-3 subtasks:\n{question}"
        response = llm_pipe(prompt)[0]["generated_text"]
        subtasks = [task.strip() for task in response.split('\n') if task.strip()]
        self.memory.write('subtasks', subtasks)

### Answer Agent
Answers each subtask using context and writes answers to shared memory.

In [5]:
class AnswerAgent:
    def __init__(self, memory):
        self.memory = memory
    def answer(self, context_provider):
        subtasks = self.memory.read('subtasks')
        answers = []
        for subtask in subtasks:
            context = context_provider(subtask)
            prompt = f"Answer this subtask using the given context.\nSubtask: {subtask}\nContext: {context}\nAnswer:"
            answer = llm_pipe(prompt)[0]["generated_text"]
            answers.append(answer)
        self.memory.write('answers', answers)

### Summarizer Agent
Summarizes all answers and writes the summary to shared memory.

In [6]:
class SummarizerAgent:
    def __init__(self, memory):
        self.memory = memory
    def summarize(self):
        answers = self.memory.read('answers')
        prompt = f"Summarize the following responses into a concise final answer:\n{' '.join(answers)}"
        summary = llm_pipe(prompt)[0]["generated_text"]
        self.memory.write('summary', summary)

### Simulated Context Provider

In [7]:
def provide_context(subtask):
    context_map = {
        "Find recent data on climate change effects.": "Global temperatures have risen by ~1.2°C since pre-industrial times...",
        "Analyze how temperature changes affect crop yields.": "Higher temperatures reduce wheat and rice yields due to heat stress...",
        "Summarize agricultural adaptation strategies.": "Farmers are adopting drought-resistant crops, precision irrigation, and agroforestry..."
    }
    return context_map.get(subtask, "No context available.")

### Run Multi-Agent System with Message Passing

In [8]:
if __name__ == "__main__":
    memory = SharedMemory()
    planner = PlannerAgent(memory)
    answerer = AnswerAgent(memory)
    summarizer = SummarizerAgent(memory)

    user_query = "Explain the impact of climate change on global agriculture."

    print("Planning Subtasks...")
    planner.plan(user_query)
    print("Subtasks:", memory.read('subtasks'))

    print("\nSolving Subtasks...")
    answerer.answer(provide_context)
    for subtask, answer in zip(memory.read('subtasks'), memory.read('answers')):
        print(f"\n✅ {subtask}\n{answer}")

    print("\nFinal Summary:")
    summarizer.summarize()
    print(memory.read('summary'))

Planning Subtasks...
Subtasks: ['The impact of climate change on global agriculture.']

Solving Subtasks...
Subtasks: ['The impact of climate change on global agriculture.']

Solving Subtasks...

✅ The impact of climate change on global agriculture.
There is no evidence that climate change has a negative impact on global agriculture.

Final Summary:

✅ The impact of climate change on global agriculture.
There is no evidence that climate change has a negative impact on global agriculture.

Final Summary:
It is not clear whether climate change has a negative impact on global agriculture.
It is not clear whether climate change has a negative impact on global agriculture.
